In [36]:
import string
import re
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from pylab import barh,plot,yticks,show,grid,xlabel,figure,cla,close
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from nltk.tokenize import WordPunctTokenizer
from scipy.sparse.linalg import svds
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2

In [5]:
df1 = pd.read_csv('data/covid19_unlabelled_test.csv')

df1

,verifiedby,country,title,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang
0,Agência Lupa,Brazil,American researcher Charles Lieber was arrest...,2020/04/06,Brazil,NaN,NaN,NaN,https://piaui.folha.uol.com.br/lupa/2020/04/06...,poynter,#Verificamos: É falso que pesquisador norte-am...,", “HOMEM QUE VENDEU O CORONA VIRUS NA CHINA, P...",NaN,pt
1,snopes,NaN,Did Trump Blame Obama for ‘Bad’ COVID-19 Tests...,2020-05-01 00:00:00,NaN,NaN,NaN,NaN,https://www.snopes.com/fact-check/trump-blame-...,snopes,Did Trump Blame Obama for 'Bad' COVID-19 Tests?,['U.S. President Donald Trump implied the Obam...,Trump and the Pandemic II,en
2,TEMPO,Indonesia,Hundreds of sampoerna cigarette factory worke...,2020/05/05,Indonesia,NaN,NaN,NaN,https://cekfakta.tempo.co/fakta/765/fakta-atau...,poynter,Tempo.co,Narasi bahwa ratusan pekerja pabrik rokok Samp...,NaN,id
3,FactCrescendo,India,Dr. Megha Vyas from Pune died of COVID-19.,2020/04/25,India,NaN,NaN,NaN,https://www.malayalam.factcrescendo.com/image-...,poynter,FACT CHECK: ചിത്രത്തില്‍ കാണുന്ന സ്ത്രി പൂനയില...,"The fact behind every news!, കോവിഡ്\u200c-19 ര...",NaN,ml
4,Full Fact,United Kingdom,Jacob Rothschild owns a patent to coronavirus.,2020/01/30,United Kingdom,NaN,NaN,NaN,fullfact.org/online/Rothschild-coronavirus/,poynter,NaN,NaN,NaN,tl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,AFP,France,President Emmanuel Macron was cheered by a cr...,2020/04/07,France,NaN,NaN,NaN,https://factuel.afp.com/non-macron-na-pas-ete-...,poynter,"Non, Macron n'a pas été applaudi à Pantin par ...","Merci à vous tous, ici à Pantin comme partout ...",NaN,fr
2088,Full Fact,United Kingdom,Five helicopters are going to spray the air w...,2020/04/01,United Kingdom,NaN,NaN,NaN,fullfact.org/health/coronavirus-helicopters/,poynter,NaN,NaN,NaN,tl
2089,Raskrinkavanje,Bosnia and Herzegovina,Coronavirus was created in a lab in order to ...,2020/01/31,Bosnia and Herzegovina,NaN,NaN,NaN,https://raskrinkavanje.ba/analiza/propolis-nij...,poynter,Propolis nije “lijek za koronavirus” - Raskrin...,Portal Aura objavio članak 31. januara u kojem...,NaN,hr
2090,Animal Político,Mexico,Video of a man dressed in white who recommend...,2020/06/18,Mexico,NaN,NaN,NaN,https://www.animalpolitico.com/elsabueso/austr...,poynter,Australia no aprobó la ivermectina u otro medi...,El video con información falsa se publicó el 1...,NaN,es


In [16]:
df2 = pd.read_csv('data/covid19_data.csv')

df2

,verifiedby,country,class,title,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang
0,Delfi Melo Detektorius (Lie Detector),Lithuania,FALSE,Claims that coronavirus is fake and Belarus i...,2020/05/11,Lithuania,NaN,NaN,NaN,https://www.delfi.lt/news/melo-detektorius/mel...,poynter,Baltarusija atšventė nesisaugodama nuo koronav...,Gegužės 9-ąją Baltarusijos sostinėje Minske su...,NaN,lt
1,AfricaCheck,"United States, South Africa",False,Muammar Gaddafi predicted the current coronav...,2020/03/21,United States,South Africa,NaN,NaN,https://africacheck.org/fbcheck/no-gaddafi-did...,poynter,"No, Gaddafi didn’t predict coronavirus, but di...",As the world struggles to bring the new corona...,NaN,en
2,AFP,Hong Kong,False,Video shows quarantined people in a building ...,2020/02/17,Hong Kong,NaN,NaN,NaN,http://u.afp.com/QuarantineChina,poynter,"This video shows tower blocks in Shanghai, not...","It has been viewed more than 200,000 times., T...",NaN,en
3,FactCrescendo,India,False,"A video shows a massive explosion in Wuhan, C...",2020/02/17,India,NaN,NaN,NaN,https://english.factcrescendo.com/2020/02/17/t...,poynter,"This blast took place in Tiajin, China in 2015...","The fact behind every news!, Ever since, the W...",NaN,en
4,Newtral.es,"Spain, Peru, United States, Mexico",False,Chlorine dioxide cures the coronavirus.,2020/03/25,Spain,Peru,United States,Mexico,https://www.newtral.es/bulo-dioxido-de-cloro-c...,poynter,El dióxido de cloro no cura el coronavirus; al...,Circula un video de Youtube titulado: “El Coro...,NaN,es
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,AFP,Argentina,MISLEADING,A video shows the Italian air force air show ...,2020/03/19,Argentina,NaN,NaN,NaN,https://factual.afp.com/el-video-del-espectacu...,poynter,El video del espectáculo aéreo de la fuerza aé...,Entradas similares circulan también en inglés....,NaN,es
5527,India Today,India,Mostly false,While coronavirus tests in other countries ar...,2020/03/25,India,NaN,NaN,NaN,https://www.indiatoday.in/fact-check/story/fac...,poynter,Fact Check: Are coronavirus tests most expensi...,"Speak Now, With more than 19,000 deaths and ov...",NaN,en
5528,TEMPO,Indonesia,Misleading,Vladimir Putin released 800 tigers andlions t...,2020/03/23,Indonesia,NaN,NaN,NaN,https://cekfakta.tempo.co/fakta/694/fakta-atau...,poynter,Tempo.co,Narasi yang menyebut Presiden Rusia Vladimir P...,NaN,id
5529,Factcheck.Vlaanderen,Belgium,misleading,The cure for coronavirus can come from Antwer...,2020/05/05,Belgium,NaN,NaN,NaN,https://factcheck.vlaanderen/factcheck/komt-de...,poynter,\n Factcheck: Komt de oplossing voor het co...,Een Antwerpse lama met de naam Winter is de ni...,NaN,nl


In [62]:
def baseline(df):
    trues, falses = 0, 0
    for val in df['class']:
        if str(val).lower() in ["correct", "true", "explanatory",\
                                "correct attribution", "news"]:
            trues += 1
        else:
            falses += 1
    print(trues, falses)
    return print("Baseline:", trues/falses)

baseline(df2)
print(df2['class'].value_counts())


79 5452
Baseline: 0.014490095377842993
False                                 2110
FALSE                                 2079
Misleading                             463
MISLEADING                             173
Mostly false                           101
Partly false                            99
misleading                              80
No evidence                             57
Mostly False                            43
Mixture                                 41
True                                    25
Explanatory                             24
No Evidence                             24
News                                    23
PARTLY FALSE                            22
Unproven                                12
MOSTLY FALSE                            11
Partly False                            10
Miscaptioned                            10
partly false                             9
mostly false                             9
Mostly True                              8
Misattributed  

In [29]:
df2.title.describe()

count                                                  5531
unique                                                 5404
top       Does Trump Own Stock in a Company CDC Uses for...
freq                                                      4
Name: title, dtype: object

In [73]:
def true_or_false(x):
    if x.lower() in ["correct", "true", "explanatory", "correct attribution", "news"]:
        x = "true"
    else:
        x = "false"
    return x


df3 = df2.copy()
df3.dropna(subset = ["title"], inplace=True)
df3.dropna(subset = ["class"], inplace=True)
df3["class"] = df3["class"].apply(str).apply(true_or_false)
df3.head()

,verifiedby,country,class,title,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang
0,Delfi Melo Detektorius (Lie Detector),Lithuania,false,Claims that coronavirus is fake and Belarus i...,2020/05/11,Lithuania,NaN,NaN,NaN,https://www.delfi.lt/news/melo-detektorius/mel...,poynter,Baltarusija atšventė nesisaugodama nuo koronav...,Gegužės 9-ąją Baltarusijos sostinėje Minske su...,NaN,lt
1,AfricaCheck,"United States, South Africa",false,Muammar Gaddafi predicted the current coronav...,2020/03/21,United States,South Africa,NaN,NaN,https://africacheck.org/fbcheck/no-gaddafi-did...,poynter,"No, Gaddafi didn’t predict coronavirus, but di...",As the world struggles to bring the new corona...,NaN,en
2,AFP,Hong Kong,false,Video shows quarantined people in a building ...,2020/02/17,Hong Kong,NaN,NaN,NaN,http://u.afp.com/QuarantineChina,poynter,"This video shows tower blocks in Shanghai, not...","It has been viewed more than 200,000 times., T...",NaN,en
3,FactCrescendo,India,false,"A video shows a massive explosion in Wuhan, C...",2020/02/17,India,NaN,NaN,NaN,https://english.factcrescendo.com/2020/02/17/t...,poynter,"This blast took place in Tiajin, China in 2015...","The fact behind every news!, Ever since, the W...",NaN,en
4,Newtral.es,"Spain, Peru, United States, Mexico",false,Chlorine dioxide cures the coronavirus.,2020/03/25,Spain,Peru,United States,Mexico,https://www.newtral.es/bulo-dioxido-de-cloro-c...,poynter,El dióxido de cloro no cura el coronavirus; al...,Circula un video de Youtube titulado: “El Coro...,NaN,es


In [74]:
train_data, test_data = train_test_split(df3, test_size=0.3, random_state=42)


vectorizer = TfidfVectorizer()#ngram_range=(1,2))
%timeit vectorizer.fit(train_data.title)

X_train = vectorizer.transform(train_data.title)
X_test = vectorizer.transform(test_data.title)

y_train = train_data['class']
y_test = test_data['class']

66.3 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [75]:
clf =  MultinomialNB()
%timeit clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print('Accuracy: {}'.format(accuracy_score(y_pred, y_test)))

6.79 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Accuracy: 0.9873417721518988
